# Data Extraction
This file contains the raw data extraction performed for our analysis of abortion demand and access. Data was pulled from the [Kaiser Family Foundation](https://www.kff.org/womens-health-policy/press-release/abortion-in-the-united-states/), the [Guttmacher Instistute](https://data.guttmacher.org/states/table) - both from the Institute's website and their repository of time series data hosted by the [OSF](https://osf.io/dashboard), and the [US Census Bureau](https://www.census.gov/quickfacts/fact/table/US/PST045222).

Data extraction methods included web scraping from KFF and the OSF, open API calls from the Guttmacher Institute, and processing of downloaded csv files from the Census Bureau.

In [1]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import re
import os
import sys
from pandas.io.json import json_normalize
from functools import reduce

os.getcwd()

'/Users/emilyjohnson/Documents/GitHub/2022-project-us_womens_health'

In [2]:
state_dict = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

## KFF Data
Pull state abortion policies from [https://www.kff.org/other/state-indicator/abortion-policy-tracker/?currentTimeframe=0] and [https://www.kff.org/womens-health-policy/state-indicator/abortion-restriction/]

In [4]:
site1 = '''https://www.kff.org/other/state-indicator/abortion-policy-tracker/?currentTimeframe=0
&sortModel=%7B"colId":"Location","sort":"asc"%7D#notes'''

r = requests.get(site1) 
soup = BeautifulSoup(r.content,'lxml')
r.close()

t = soup.find_all("script", attrs={'type':"text/javascript"})
for ti in t:
    sp = ti.get_text()
    if("Status of Abortion" in sp):
        state_policies = sp
        
state_policies = state_policies.split("[[")[2].split("],[")
state_policies = state_policies[3:-1]

states = pd.DataFrame(columns = ["state","policy","notes"])
for s in state_policies:
    s = s.split('","')
    states.loc[len(states)] = s
    
states["state"] = states["state"].str[1:]
states["notes"] = states["notes"].str[:-2]
states["state"] = states["state"].map(state_dict)

states.head()

,state,policy,notes
0,AL,Abortion banned,"State abortion ban went into effect June 24, 202"
1,AK,Abortion legal beyond 22 weeks LMP,No state law protecting or banning abortion; T...
2,AZ,Gestational limit between 15 and 22 weeks LMP,Current gestational limit is 15 weeks LMP. Pre...
3,AR,Abortion banned,"Trigger law went into effect June 24, 2022, af..."
4,CA,Abortion legal beyond 22 weeks LMP,State law protects the right to abortio


In [5]:
site = '''https://www.kff.org/womens-health-policy/state-indicator/abortion-restriction/?current
Timeframe=0&sortModel=%7B"colId":"Location","sort":"desc"%7D'''

r = requests.get(site) 
soup = BeautifulSoup(r.content, 'lxml')
r.close()

t = soup.find_all("script", attrs={'type':"text/javascript"})
for ti in t:
    ip = ti.get_text()
    if("Insurance" in ip):
        insurance_policies = ip
        
insurance_policies = insurance_policies.split("[[")[2].split("],[")
insurance_policies = insurance_policies[3:-1]

insurance = pd.DataFrame(columns = ['state','private_insurance','marketplace','public_employee'])
for s in insurance_policies:
    s = s.split('","')
    insurance.loc[len(insurance)] = s
    
insurance["state"] = insurance["state"].str[1:]
insurance["public_employee"] = insurance["public_employee"].str[:-1]
insurance["state"] = insurance["state"].map(state_dict)

insurance.head()

states = states.merge(insurance, on = "state")
states.to_csv("data/current_state_policies.csv", index = False)
states.head()

,state,policy,notes,private_insurance,marketplace,public_employee
0,AL,Abortion banned,"State abortion ban went into effect June 24, 202",No,Yes,No
1,AK,Abortion legal beyond 22 weeks LMP,No state law protecting or banning abortion; T...,No,No,No
2,AZ,Gestational limit between 15 and 22 weeks LMP,Current gestational limit is 15 weeks LMP. Pre...,No,Yes,Yes
3,AR,Abortion banned,"Trigger law went into effect June 24, 2022, af...",No,Yes,No
4,CA,Abortion legal beyond 22 weeks LMP,State law protects the right to abortio,No,No,No


## OSF Data
Pull abortion time series data by state from [https://osf.io/kthnf/?view_only=]

In [6]:
r = requests.get('https://osf.io/download/td7mk/')
stats = r.text
r.close()

stats = stats.split("\n")
headers = stats[0].split(",")

stat_df = pd.DataFrame(columns = headers)
for row in stats[1:913]:
    row = re.split(''',(?=(?:[^'"]|'[^']*'|"[^"]*")*$)''', row)
    stat_df.loc[len(stat_df)] = row

abortion_columns = stat_df.columns[stat_df.columns.str.contains("abortion")]
stat_df = stat_df[stat_df.columns[stat_df.columns.str.contains("abortion|state|year")]]
stat_df = pd.melt(stat_df, id_vars = ["state","year"])
stat_df['age'] = stat_df['variable'].str.replace("abortionrate|abortionratio|abortions", "", regex = True)
stat_df['variable'] = stat_df['variable'].str.extract("(abortionrate|abortionratio|abortions)")

stat_df = pd.pivot(stat_df, index = ['state','year','age'], columns = "variable", values = "value")
stat_df.reset_index(inplace=True)
stat_df.replace("NA", pd.NA, inplace=True)
stat_df['year'] = pd.to_numeric(stat_df['year'])
stat_df['abortions'] = pd.to_numeric(stat_df['abortions'])
stat_df['abortionrate'] = pd.to_numeric(stat_df['abortionrate'])
stat_df['abortionratio'] = pd.to_numeric(stat_df['abortionratio'])
stat_df.drop('abortionratio', axis = 1, inplace = True)

stat_df.to_csv("data/abortions_over_time_us.csv", index = False)
stat_df.head()

## Guttmacher Website Data
Pull multiple metrics related to insurance and abortion access from [https://data.guttmacher.org/states]

In [8]:
site = '''https://api.guttmacher.org/measure/data/283+188+162+228+161+78+79+80+81+197+198+88?
&visualization=datatable&dataset=data'''

rq = requests.get(site)
guttmacher_json = json.loads(rq.text)
metadata = pd.json_normalize(guttmacher_json['response']['measure'])

guttmacher = []
for data in guttmacher_json['response']['data']:
    df = pd.json_normalize(data)
    df = df[['measure_id','state_id','datum']]

    # Get metadata relevant to metric of interest
    measure = df.measure_id.drop_duplicates()[0]
    met = metadata.loc[metadata['id'] == measure].to_dict('list')
    measure_name = met['name'][0] + ", " + str(met['years'][0][0])
    if(met['number_format'] == "percentage"):
        df['datum'] /= 100
    df = df[['state_id','datum']]
    df.rename(columns={'datum':measure_name}, inplace=True)
    guttmacher.append(df)

guttmacher = reduce(lambda x, y: pd.merge(x, y, on = 'state_id'), guttmacher).drop_duplicates()

guttmacher.replace("n/a", pd.NA, inplace=True)
guttmacher[['No. of abortions, by state of residence, 2020','% change in abortion rate, 2017-2020']] = \
    guttmacher[['No. of abortions, by state of residence, 2020',
                '% change in abortion rate, 2017-2020']].apply(pd.to_numeric)

guttmacher.to_csv("data/abortion_access_data.csv", index = False)
guttmacher.head()


,state_id,"% change in abortion rate, 2017-2020","% of women aged 15-44 covered by Medicaid, 2017","% of women aged 15-44 covered by private insurance, 2017","% of women aged 15-44 who are uninsured, 2017","No. of abortions per 1,000 women aged 15–44, by state of occurrence, 2020","No. of abortions per 1,000 women aged 15–44, by state of residence, 2020","No. of abortions, by state of occurrence, 2020","No. of abortions, by state of residence, 2020","No. of federally funded abortions, 2010","No. of state funded abortions, 2010","Total no. of publicly funded abortions , 2010"
0,AL,-6,18,63,14,6.0,9.5,5700,9060.0,9,0,9
3,AK,0,24,49,15,8.6,9.2,1240,1320.0,0,835,835
6,AZ,1,24,60,13,9.3,9.7,13320,13820.0,1,13,14
9,AR,2,28,58,11,5.6,7.8,3250,4510.0,0,0,0
12,CA,17,27,62,9,19.2,19,154060,152400.0,0,88466,88466


## Census Bureau
Process current state population data from the US Census Bureau

In [3]:
df = pd.read_excel('data/NST-EST2022-POP.xlsx')
state_data = df.iloc[8:59, [0, 4]]
state_data.columns = ['state', 'population']
state_data.to_csv('data2/state_census_data.csv', index=False)

In [5]:
census = pd.read_csv('data2/state_census_data.csv')
census['state'] = census['state'].str.replace('.', '', regex = True)
census['state'] = census['state'].map(state_dict)
census.to_csv('data/census_updated.csv', index=False)
census.head()

,state,population
0,.Alabama,5074296.0
1,.Alaska,733583.0
2,.Arizona,7359197.0
3,.Arkansas,3045637.0
4,.California,39029342.0
